In [ ]:
import pandas as pd
import numpy as np
from indicnlp.tokenize import indic_tokenize
from transformers import AutoTokenizer
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-T5/T5-base")

In [ ]:
data = pd.read_csv("Sentence pairs in English-Hindi - 2025-02-11.tsv",sep="\t",header=None,
                   names=["SrcSentenceID","SrcSentence","DstSentenceID","DstSentence"])

In [ ]:
data.head()

In [ ]:
data["DstSentence"] = data["DstSentence"].apply(lambda x: indic_tokenize.trivial_tokenize(x,lang="hi"))

In [ ]:
data["SrcSentence"] = data["SrcSentence"].apply(tokenizer.tokenize)

In [ ]:
data.head()

In [ ]:
data["SrcSentence"] = data["SrcSentence"].apply(tokenizer.convert_tokens_to_ids)

In [ ]:
Vs = tokenizer.get_vocab()

In [ ]:
Vd = set()
for tokenized_hindi_sentence in data["DstSentence"]:
    Vd.update(tokenized_hindi_sentence)

hindi_vocab = dict()
for idx, token in enumerate(Vd):
    hindi_vocab[token] = idx + 3

hindi_vocab["<PAD>"] = 0
hindi_vocab["<SOS>"] = 1
hindi_vocab["<EOS>"] = 2

Vd = hindi_vocab

In [ ]:
data.head()

In [ ]:
def convert_hindi_tokens_to_ids(tokenized_hindi_sentence):
    return [Vd[token] for token in tokenized_hindi_sentence]

In [ ]:
data["DstSentence"] = data["DstSentence"].apply(convert_hindi_tokens_to_ids)

In [ ]:
data.head()

In [ ]:
def insert_sos_token_id(hindi_sentence_token_ids_list):
    return [1] + hindi_sentence_token_ids_list

In [ ]:
def insert_eos_token_id(hindi_sentence_token_ids_list):
    return hindi_sentence_token_ids_list + [2]

In [ ]:
data["DstSentenceInput"] = data["DstSentence"].apply(insert_sos_token_id)
data["DstSentenceLabel"] = data["DstSentence"].apply(insert_eos_token_id)

In [ ]:
data.head()

In [ ]:
data.drop(labels=["SrcSentenceID","DstSentenceID","DstSentence"],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
X = list(data["SrcSentence"])
Y_input = list(data["DstSentenceInput"])
Y_label = list(data["DstSentenceLabel"])

X_tensor = [torch.tensor(eng_tokenized_ids) for eng_tokenized_ids in X]
Y_input_tensor = [torch.tensor(hin_tokenized_ids) for hin_tokenized_ids in Y_input]
Y_label_tensor = [torch.tensor(hin_tokenized_ids) for hin_tokenized_ids in Y_label]

X_padded = torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)
Y_input_padded = torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)
Y_label_padded = torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [ ]:
Ns = X_padded.shape[1]
Nd = Y_label_padded.shape[1]

In [ ]:
class Attention(torch.nn.Module):
    def __init__(self):
        super(Attention,self).__init__()
        self.attention_probabilities = torch.nn.Softmax(dim=1)

    def forward(self,encoder_outputs,decoder_lstm_layer_outputs):
        
        decoder_lstm_layer_outputs = torch.transpose(decoder_lstm_layer_outputs,dim0=1,dim1=2)
        alignment_scores = torch.bmm(encoder_outputs,decoder_lstm_layer_outputs)
        attention_weights = self.attention_probabilities(alignment_scores)
        attention_weights = torch.transpose(attention_weights,dim0=1,dim1=2)
        context_vectors = torch.bmm(attention_weights,encoder_outputs)

        return context_vectors

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self,src_lang_vocab_size,topic_vector_dim):
        super(Encoder,self).__init__()
        self.first_emebdding_layer = torch.nn.Embedding(num_embeddings=src_lang_vocab_size,
                                                        embedding_dim=topic_vector_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=topic_vector_dim,hidden_size=topic_vector_dim,
                                               batch_first=True)
        
    def forward(self,X_padded_mini_batch):

        first_embedding_layer_out = self.first_emebdding_layer(X_padded_mini_batch)
        encoder_outputs,(final_encoder_output,final_candidate_cell_state) = self.second_lstm_layer(first_embedding_layer_out)

        return encoder_outputs,(final_encoder_output,final_candidate_cell_state)

In [ ]:
class Decoder(torch.nn.Module):
    def __init__(self,dst_lang_vocab_size,topic_vector_dim):
        super(Decoder,self).__init__()
        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=dst_lang_vocab_size,
                                                        embedding_dim=topic_vector_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=topic_vector_dim,hidden_size=topic_vector_dim,
                                               batch_first=True)
        self.attention_layer = Attention()
        self.output_layer = torch.nn.Linear(in_features=topic_vector_dim*2,out_features=dst_lang_vocab_size)
        self.output_layer_activation = torch.nn.Softmax(dim=2)

    def forward(self,encoder_outputs,initial_hidden_state,initial_candidate_cell_state,
                Y_padded_mini_batch):

        first_embedding_layer_out = self.first_embedding_layer(Y_padded_mini_batch)
        decoder_lstm_layer_outputs,final_cell_hidden_states = self.second_lstm_layer(first_embedding_layer_out,
                                                                                    (initial_hidden_state,
                                                                                    initial_candidate_cell_state))
        context_vectors = self.attention_layer(encoder_outputs,decoder_lstm_layer_outputs)
        concatenated_lstm_layer_output = torch.concatenate(tensors=(decoder_lstm_layer_outputs,context_vectors),dim=2)
        affine_transformed_output = self.output_layer(concatenated_lstm_layer_output)
        decoder_outputs = self.output_layer_activation(affine_transformed_output)

        return decoder_outputs

In [ ]:
class Seq2SeqEncDecWithAttn(torch.nn.Module):
    def __init__(self,src_lang_vocab_size,dst_lang_vocab_size,topic_vector_dim):
        super(Seq2SeqEncDecWithAttn,self).__init__()
        self.encoder = Encoder(src_lang_vocab_size,topic_vector_dim)
        self.decoder = Decoder(dst_lang_vocab_size,topic_vector_dim)

    def forward(self,X_padded_mini_batch,Y_padded_mini_batch_input):

        encoder_outputs,(final_encoder_output,final_candidate_cell_state) = self.encoder(X_padded_mini_batch)
        Y_hat_mini_batch = self.decoder(encoder_outputs,final_encoder_output,final_candidate_cell_state,
                                       Y_padded_mini_batch_input)
        
        return Y_hat_mini_batch

In [ ]:
X_padded_train = X_padded[0:13000]
Y_input_padded_train = Y_input_padded[0:13000]
Y_label_padded_train = Y_label_padded[0:13000]

X_padded_test = X_padded[13000:]
Y_input_padded_test = Y_input_padded[13000:]
Y_label_padded_test = Y_label_padded[13000:]

In [ ]:
nw = Seq2SeqEncDecWithAttn(src_lang_vocab_size=len(Vs),dst_lang_vocab_size=len(Vd),topic_vector_dim=32)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(params=nw.parameters())
epochs = 5
mb_size = 26

for epoch in range(epochs):
    for i in range(X_padded_train.shape[0]//mb_size):

        X_train_mb = X_padded_train[i*mb_size:(i+1)*mb_size]
        Y_input_train_mb = Y_input_padded_train[i*mb_size:(i+1)*mb_size]
        Y_label_train_mb = Y_label_padded_train[i*mb_size:(i+1)*mb_size]

        Y_label_train_mb = Y_label_train_mb.reshape(Y_label_train_mb.shape[0]*Y_label_train_mb.shape[1],)

        Y_pred_train_mb = nw(X_train_mb,Y_input_train_mb)
        Y_pred_train_mb = Y_pred_train_mb.reshape(Y_pred_train_mb.shape[0]*Y_pred_train_mb.shape[1],
                                                  Y_pred_train_mb.shape[2])
        

        loss_fn_value = loss_fn(Y_pred_train_mb,Y_label_train_mb)

        loss_fn_value.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 25 == 0:
            print("Epoch #{}, Mini Batch #{}, CCE Loss = {}".format(epoch,i,loss_fn_value))